# 👗 FashionMNIST cGAN 미션 정리

## 🎯 미션 소개
이번 미션에서는 **cGAN (Conditional GAN)** 모델을 활용하여  
**FashionMNIST 데이터셋의 각 패션 아이템(예: 티셔츠, 바지, 스니커즈 등)** 을  
조건부로 생성하는 작업을 수행합니다.

- 각 클래스(Label)에 해당하는 이미지를 직접 생성하는 **조건부 생성 모델**을 설계하고 학습시켜 보세요.
- Generator와 Discriminator 모두 직접 구현하고, 학습 과정을 시각화하며 결과를 분석합니다.

---

## 🧺 데이터 소개

| 항목 | 내용 |
|------|------|
| **데이터 링크** | `torchvision.datasets.FashionMNIST` 를 통해 바로 다운로드 가능 |
| **훈련 데이터** | 60,000장 |
| **테스트 데이터** | 10,000장 |
| **이미지 크기** | 28×28 픽셀 (흑백) |
| **클래스 수** | 10개 클래스 |

### 👕 클래스 목록
1. T-shirt/top  
2. Trouser  
3. Pullover  
4. Dress  
5. Coat  
6. Sandal  
7. Shirt  
8. Sneaker  
9. Bag  
10. Ankle boot

> ⚠️ 참고: 문서에 “GURU”가 포함되어 있으나 실제 FashionMNIST에는 존재하지 않습니다.

---

## 🧪 분석 과정 및 필수 포함 내용

1. **데이터 로드 및 전처리**  
   - `torchvision.datasets.FashionMNIST` 활용  
   - 정규화 및 텐서 변환 수행  

2. **모델 설계 (Conditional GAN 구현)**  
   - 조건(label) + 노이즈(z)를 입력으로 받아 이미지 생성  
   - Generator와 Discriminator 직접 정의  

3. **모델 학습**  
   - 훈련 과정(loss 변화) 시각화  
   - 각 클래스별 생성 이미지 확인  

4. **생성 이미지 예측 및 평가**  
   - 조건(label)을 주어 각 클래스별 패션 이미지 생성  
   - 시각적(qualitative) 및 정량적(quantitative) 평가 수행  

5. **결과 시각화 및 보고**  
   - `matplotlib` 또는 `torchvision.utils.make_grid`로 생성 결과 표시  
   - 학습된 모델의 성능 및 한계에 대한 분석 포함  

---

## 🧾 마크다운 설명 가이드
- 각 코드 셀의 **의도, 알고리즘, 함수 설명**을 마크다운으로 자세히 기록하세요.  
- 코드와 결과가 한눈에 들어올 수 있도록 셀 순서를 논리적으로 구성합니다.

---

## 🧮 모델 성능 평가
- **정성적 평가 (Qualitative)**  
  - 클래스별 생성 이미지의 시각적 품질 비교  
- **정량적 평가 (Quantitative, 선택 사항)**  
  - FID(Fréchet Inception Distance), IS(Inception Score) 등 계산  

---

## 🧠 모델 성능 개선 아이디어

### 🧱 1. 데이터 증강 (Data Augmentation)
- **목적:** 이미지 다양성을 높여 일반화 성능 향상  
- **방법 예시:**  
  - 회전(`RandomRotation`)  
  - 크롭(`RandomCrop`)  
  - 밝기/대비 조절(`ColorJitter`)  
  - 좌우 반전(`RandomHorizontalFlip`)

```python
transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # -1~1 정규화
])

# 사전 설정

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.utils as utils
import matplotlib.pyplot as plt
import numpy as np
import random
import os

In [3]:
#@title Device 설정

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
#@title Drive Mount
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/study/data'

Mounted at /content/drive


In [5]:
#@title 일정한 결과값 확인을 위한 seed설정

seed = 42
torch.manual_seed(seed)                 # torch 모든 연산 난수 고정
torch.cuda.manual_seed_all(seed)        # cuda난수 고정 (cuda여러개여도 전부 적용)
random.seed(seed)                       # random모듈 난수 고정
np.random.seed(seed)                    # numpy 난수 고정

# 데이터 전처리

In [6]:
#@title 하이퍼파라미터 및 환경 설정

image_size = 28             # Fashion MNIST 이미지 크기
num_classes = 10            # 10개 클래스
latent_dim = 100            # 잠재 공간 차원
batch_size = 64             # 학습 배치 크기
num_epochs = 200            # 전체 학습 에폭 수
lr = 0.0002                 # 학습률: GAN 학습 안정화를 위해 일반적으로 낮은 값 사용
beta1 = 0.5                 # Adam optimizer beta1: DCGAN 논문에서 제안된 값으로 GAN 학습 안정화에 도움
beta2 = 0.999               # Adam optimizer beta2: Adam optimizer의 기본값

In [7]:
#@title Transform 설정

# 학습용 데이터 증강 transform
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.2),                         # 20% 확률로 좌우 반전
    transforms.RandomRotation(10),                                  # ±10도 범위 내에서 랜던 회전
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),                   # 최대 10% 이동
    transforms.RandomResizedCrop(28, scale=(0.8, 1.0), ratio=(0.9, 1.1)),       # 랜덤 크롭 후 28X28로 리사이즈
    transforms.ToTensor(),                                                      # [0,255] 픽셀 → [0.0,1.0] float 텐서(C×H×W)
    transforms.Normalize((0.5,), (0.5,))                                        # (x-0.5)/0.5 → [-1,1]로 정규화(그레이스케일 1채널)
])

# 평가용 transform (증강없이 정규화만)
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [8]:
#@title Dataset + DataLoader

train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=train_transforms)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=test_transforms)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

100%|██████████| 26.4M/26.4M [00:02<00:00, 9.79MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 155kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 2.94MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 28.0MB/s]


In [9]:
#@title 클래스 이름 설정
idx_to_class = {i: class_name for i, class_name in enumerate(train_dataset.classes)}
print("FashionMNIST classes:", idx_to_class)

FashionMNIST classes: {0: 'T-shirt/top', 1: 'Trouser', 2: 'Pullover', 3: 'Dress', 4: 'Coat', 5: 'Sandal', 6: 'Shirt', 7: 'Sneaker', 8: 'Bag', 9: 'Ankle boot'}


# 모델 생성

In [10]:
#@title Generator Model

# Generator: label과 noise로 패션이미지 만들기
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        # 레이블 임베팅: 숫자(0~9)를 벡터(10차원)로 변환해요
        self.label_emb = nn.Embedding(num_classes, num_classes)

        # 초기 특성 맵 크기 계산(28X28 >> 7X7)
        self.init_size = image_size // 4        # 28/4 = 7

        # 첫번째 Linear layer 입력: 노이즈(100차원) + 라벨 벡터(10차원) 출력: 128채널 × 7 × 7 (총 6272개 값)
        self.l1 = nn.Sequential(
            nn.Linear(latent_dim + num_classes, 128 * self.init_size * self.init_size),
            nn.ReLU(inplace=True)               # ReLU로 빈선형성 추가
        )

        # Conv 블록: 7X7 특징 맵을 점점 키워서 28X28 이미지로 만들기
        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128, momentum=0.89),                         # 채널 정규화 >> 학습 안정화
            nn.Upsample(scale_factor=2),                                # 크기 2배
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),     # 세부 패턴 만들기
            nn.BatchNorm2d(128, momentum=0.8),                          # 채널 수 128에 대한 정규화
            nn.ReLU(inplace=True),

            nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),     # 채널 줄이기
            nn.BatchNorm2d(64, momentum=0.8),
            nn.ReLU(inplace=True),

            nn.Upsample(scale_factor=2),                                # 크기 2배
            nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32, momentum=0.8),
            nn.ReLU(inplace=True),

            nn.Conv2d(32, 1, kernel_size=3, stride=1, padding=1),       # 흑백 이미지 1채널
            nn.Tanh()                                                   # 출력 범위 [-1, 1] → Normalize된 실제 이미지와 일치시킴
        )

    def forward(self, noise, labels):
        # 입력된 노이즈 +라벨로 이미지를 생성
        # 숫자라벨 >> 벡터 변환
        label_input = self.label_emb(labels)
        # 노이즈 100차원과 라벨벡터10차원을 합치기
        gen_input = torch.cat((noise, label_input), -1)
        # Linear Layer 통과
        out = self.l1(gen_input)
        # 2D Tensor로 변환 (batch_size, 128, 7, 7)
        out = out.view(out.size(0), 128, self.init_size, self.init_size)
        # Conv 블록 통과 >> 28X28 이미지 생성
        img = self.conv_blocks(out)

        return img

In [11]:
#@title Discriminator Model

# Discriminator: 이미지가 "진짜인지 가짜인지" 판변
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        # 레이블 임베딩: 각 클래스 번호 (0~9)를 실수값으로 바꾸기//여기서는 단일 채널로 표현 즉, 한 장짜리 "라벨 이미지"를 만든다고 생각
        self.label_emb = nn.Embedding(num_classes, 1)

        # 이미지+라벨을 함께 받아서판별하는 신경망 구조
        self.model = nn.Sequential(
            nn.Conv2d(1 + 1, 64, kernel_size=3, stride=2, padding=1),           # 입력: 1+1 2채널 출력: 64 커널 3x3, stride2 28>>14
            nn.LeakyReLU(0.2, inplace=True),                                    # 약간의 음수도 통과

            # 64채널 >> 128채널로 늘림, 공간 크기는 절반(14>>7)
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(128),                                                # 채널 정규화 (학습안정화)
            nn.LeakyReLU(0.2, inplace=True),

            # Flatten + Linear
            nn.Flatten(),                                                      # 2D 텐서를 1D로 펼침
            nn.Linear(128 * (image_size // 4) * (image_size // 4), 1),          # 마지막에 1개의 확률로 출력
            nn.Sigmoid()                                                        # 결과를 [0, 1]사이로 (1:real, 0:fake)
        )

    def forward(self, img, labels):
        batch_size = img.size(0)
        # 숫자 라벨 >> 실수값 변환
        label = self.label_emb(labels)
        # 레이블 이미지 크기와 맞게 확장
        label = label.view(batch_size, 1, 1, 1)
        label = label.expand(batch_size, 1, image_size, image_size)
        # 이미지와 레이블을 채널 차원에서 연결
        d_in = torch.cat((img, label), 1)
        # 모델통과>>진짜/가짜 확률 산출
        validity = self.model(d_in)
        return validity

In [12]:
#@title 모델 초기화 및 optimizer 설정

generator = Generator().to(device)
discriminator = Discriminator().to(device)
adversarial_loss = nn.BCELoss()         # Binary Cross Entropy Loss: GAN 학습에 사용되는 손실 함수

# Generator를 위한 Adam optimizer 설정
optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(beta1, beta2))
# Discriminator를 위한 Adam optimizer 설정
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, beta2))

## 모델 학습

In [13]:
#@title 한 에폭 학습 함수

def train_one_epoch(generator, discriminator,
                    optimizer_G, optimizer_D,
                    adversarial_loss, train_loader,
                    device, latent_dim, num_classes):
    """
    한 에폭 동안 Generator•Discriminator 학습하고
    평균 손실 및 판별자 출력 통계변환
    """
    # 손실값과 판별값 누적용 변수 초기화
    g_loss_sum = d_loss_sum = d_real_sum = d_fake_sum = 0.0
    num_batches = 0

    # 모델을 '학습 모드'로 전환
    generator.train()
    discriminator.train()

    # 데이터로더에서 이미지와 레이블 가져오기
    for imgs, labels in train_loader:
        batch_size = imgs.size(0)
        imgs, labels = imgs.to(device), labels.to(device)

        # 진짜/가짜 판변 레이블 생성
        valid = torch.ones(batch_size, 1, device=device)   # 진짜 1
        fake = torch.zeros(batch_size, 1, device=device)    # 가짜 0

        # 노이즈 라벨을 이용해 Generator가 가짜이미지 생성
        z = torch.randn(batch_size, latent_dim, device=device)                  # 랜덤 노이즈 벡터
        gen_labels = torch.randint(0, num_classes, (batch_size,), device=device)# 생성할 이미지의 랜덤레이블 생성
        gen_imgs = generator(z, gen_labels)                                     # 노이즈와 레이블을 이용해 가짜 이미지 생성

        # Discriminator 학습 단계
        optimizer_D.zero_grad()
        real_validity = discriminator(imgs, labels)                             # 진짜 이미지 판별
        d_real_loss = adversarial_loss(real_validity, valid)                    # 진짜 이미지를 진짜로 판별하도록 유도

        fake_validity = discriminator(gen_imgs.detach(), gen_labels)            # 생성된 가짜이미지를 Discriminator에 입력하여 판별 결과 얻기
        d_fake_loss = adversarial_loss(fake_validity, fake)                     # 가짜이미지를 0으로 예측하도록 유도

        d_loss = 0.5 * (d_real_loss + d_fake_loss)                              # Discriminator 전체 손실 계산
        d_loss.backward()
        optimizer_D.step()

        # Generator 학습 단계
        optimizer_G.zero_grad()
        validity = discriminator(gen_imgs, gen_labels)                          # Generator가 만든이미지르 Discriminator에 다시 입력
        g_loss = adversarial_loss(validity, valid)                              # Discriminator가 "가짜를 진짜로" 착각하도록 학습
        g_loss.backward()                                                       # Generator의 기울기 계산
        optimizer_G.step()                                                      # Generator 가중치 업데이트

        # 손실 및 판별 결과 누적
        g_loss_sum += g_loss.item()                                             # Generator 손실 합산
        d_loss_sum += d_loss.item()                                             # Discriminator 손실 합산
        d_real_sum += d_real_loss.item()                                        # 진짜 이미지 판별 손실 합산
        d_fake_sum += d_fake_loss.item()                                        # 가짜 이미지 판별 손실 합산
        num_batches  += 1

    # 한 에폭의 평균 손실 및 통계 계산
    return {
        "g_loss": g_loss_sum / num_batches,         # Generator 평균 손실
        "d_loss": d_loss_sum / num_batches,         # Discriminator 평균 손실
        "d_real": d_real_sum / num_batches,         # 진짜에 대한 예측 평균값
        "d_fake": d_fake_sum / num_batches          # 가짜에 대한 예측 평균값
    }

In [14]:
#@title 이미지 생성·시각화 함수

@torch.no_grad()    # 그래디언트 계산 비활성화
def visualize_generated_images(generator, epoch,
                               latent_dim, num_classes,
                               device, idx_to_class,
                               n_row=3, show_every=10):
    """
    지정된 주기(show_every)마다 클래스별 n_row개의 이미지를 생성하고 시각화 한다.
    """

    # 지정한 주기가 아니면 아무것도 하지 않고 종료
    if epoch % show_every != 0:
        return

    # Generator 평가 모드 전환
    generator.eval()

    n_col = num_classes     # 열의 개수는 클래스 개수와 동일

    # 각 클래스별 n_row개의 샘플을 만들기 위한 라벨 텐서 생성
    labels = torch.arange(num_classes, device=device).repeat(n_row, 1).view(-1)

    # 생성할 이미지 개수만큼 노이즈 백터(z) 생성
    z = torch.randn(n_row * n_col, latent_dim, device=device)

    # Generator 사용하여 이미지 생성
    gen_imgs = generator(z, labels)
    gen_imgs = (gen_imgs + 1) / 2.0         # 이미지 정규화 해제

    # subplot(row: n_row, col: n_col)으로 이미지 표시
    fig, axs = plt.subplots(n_row, n_col, figsize=(n_col, n_row), squeeze=False)

    # 생성된 이미지를 subplot에 표시
    for i in range(n_row):
        for j in range(n_col):
            idx = i * n_col + j
            axs[i, j].imshow(gen_imgs[idx, 0].cpu(), cmap="gray")           # 흑백으로 표시
            axs[i, j].axis('off')
            # 첫 번째 행에만 클래스 이름 표시
            if i == 0:
                axs[i, j].set_title(idx_to_class[j], fontsize=9)

    plt.tight_layout()
    plt.show()

In [15]:
#@title 전체 학습 루프 함수

def train_cgan(num_epochs, generator,
               discriminator, optimizer_G,
               optimizer_D, adversarial_loss,
               train_loader, device, latent_dim,
               num_classes, idx_to_class):
    """
    전체 cGAN 학습을 수행하는 메인 루프 함수
    - 한 epoch마다 train_one_epoch()로 G/D학습
    - 손실값 출력
    - 일정 주기마다 임지 시각화
    - 모델 가중치 저장
    """

    # 지정된 epoch수만큼 학습 반복
    for epoch in range(1, num_epochs + 1):
        # 편균 손실과 D의 출력 통걔 반환
        stats = train_one_epoch(generator, discriminator,
                                optimizer_G, optimizer_D,
                                adversarial_loss, train_loader,
                                device, latent_dim, num_classes)

        # 현재 에폭의 학습 결과 출력
        print(f"Epoch [{epoch}/{num_epochs}] "
              f"G_loss: {stats['g_loss']:.4f}  "                # Generator가 얼마나 잘 속는지
              f"D_loss: {stats['d_loss']:.4f}  "                # Discriminator가 얼마나 잘 구분하는지
              f"D(x) : {stats['d_real']:.4f}  "                 # 진짜 이미지에 대한 판별기 출력 평균(1에 가까울수록 좋다)
              f"D(G(x)) : {stats['d_fake']:.4f}")               # 가짜 이미지에 대한 판별기 출력 평균(0에 가까울수록 좋다)

        # 지정도니 주기마다 Generator가 만든 이미지 시각화
        visualize_generated_images(generator, epoch, latent_dim,
                                   num_classes, device, idx_to_class,
                                   n_row=3, show_every=10)              # show_every=10 >> 10, 20, 30번째 epoch마다 출력

        # 매 epoch 마다 저장
        torch.save(generator.state_dict(), os.path.join(path, f"generator_tr_epoch_{epoch}.pth"))
        torch.save(discriminator.state_dict(), os.path.join(path, f"discriminator_tr_epoch_{epoch}.pth"))
        print(f"Saved models for epoch {epoch}")

In [17]:
#@title 학습 실행

train_cgan(
    num_epochs=num_epochs,                                      # 학습 반복 횟수
    generator=generator.to(device),
    discriminator=discriminator.to(device),
    optimizer_G=optimizer_G,
    optimizer_D=optimizer_D,
    adversarial_loss=torch.nn.BCELoss(),
    train_loader=train_loader,
    device=device,
    latent_dim=latent_dim,                                      # 노이즈 벡터 차원
    num_classes=num_classes,
    idx_to_class=idx_to_class,                                  # 숫자라벨 >> 클래스명 변환 딕셔너리
)

Output hidden; open in https://colab.research.google.com to view.

# 모델 평가

In [21]:
#@title 학습된 모델 로드

# Generator 모델을 다시 인스턴스화 합니다.
generator_loaded = Generator().to(device)

# 정장된 state_dict를 로드합니다. 파일 경로를 정확하게 지정해야 합니다.
model_path = os.path.join(path, f"generator_tr_epoch_{num_epochs}.pth")
generator_loaded.load_state_dict(torch.load(model_path, map_location=device))
generator_loaded.eval()     # 평가 모드로 설정 (Dropout, batchNorm 등이 평가 모드로 동작)

print(f"Generator model loaded from {model_path}")

Generator model loaded from /content/drive/MyDrive/study/data/generator_tr_epoch_200.pth


In [23]:
#@title Inception model 준비
import torch.nn.functional as F
from torchvision.models import inception_v3
from scipy.linalg import sqrtm
import numpy as np

# Inception_v3 모델 사용하여 사전 학습된 이미지넷 가중치 사용
# - 생성된 이미지 품질 평가에 필요한 feature 추출용 네트워크

# Inception‑v3 모델 준비 (FID용: fc를 Identity로 변경) : 특징 백터만 필요하고, 분류기 부분은 필요없음
inception_model_fid = inception_v3(weights="IMAGENET1K_V1", transform_input=False).to(device)
# 최종 Fully connected layer를 identity로 대체 : 분류를 생략하고 2048차원 feature백터를 그대로 반환하도록 함
inception_model_fid.fc = torch.nn.Identity()
# 평가모드 전환
inception_model_fid.eval()

# Inception score 계산용 모델
inception_model_is = inception_v3(weights="IMAGENET1K_V1", transform_input=False).to(device)
inception_model_is.eval()

print("Inception meodel loaded")

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth


100%|██████████| 104M/104M [00:00<00:00, 205MB/s] 


Inception meodel loaded


- Fréchet Inception Distance

    - 생성 분포와 실제 데이터 분포간의 거리 측정
    - 거리가 가까울수록 (FID 값이 작을 수록) 실제 분포와 유사한 데이터가 생성


- FID는 "생성된 이미지의 특징 분포가 실제 이미지의 특징 분포와 얼마나 비슷한가?"를 측정하는 지표입니다.

- 실제 이미지와 생성 이미지를 직접 비교하며, 점수가 낮을수록 좋습니다.

- 'Distance(거리)'라는 이름처럼, 두 그룹 간의 거리가 가까울수록 좋다는 의미입니다.

- 계산 순서

    1. 수천 장의 실제 이미지와 모델이 생성한 이미지를 준비합니다.

    2. 두 이미지 그룹을 모두 Inception-v3 모델에 통과시켜 특징 벡터(Feature Vector)를 추출

    3. '실제 이미지 특징 벡터 그룹'과 '생성된 이미지 특징 벡터 그룹' 각 그룹의 통계적 특성(평균과 공분산)을 계산

    5. 프레셰 거리(Fréchet Distance)라는 공식을 사용해 두 그룹의 평균과 공분산이 얼마나 차이 나는지를 계산

In [65]:
#@title Inception Feature 추출 함수 + Fréchet Inception Distance (FID) 계산 함수

# 입력 이미지는 [0,1] 범위, 크기 28x28, 채널: 1 (grayscale)
# Inception‑v3는 (3,299,299) 크기의 이미지를 요구하므로, 변환 과정을 포함.
def get_inception_features(images, model, batch_size=32, device=device):
    """
    Inception-v3 모델을 사용하여 이미지에서 특징 추출
    """
    model.eval() # 모델을 평가 모드로 설정
    n_images = images.shape[0]
    features = []
    # Inception‑v3의 입력 정규화 값 (ImageNet)
    norm_mean = torch.tensor([0.485, 0.456, 0.406], device=device).view(1,3,1,1)
    norm_std  = torch.tensor([0.229, 0.224, 0.225], device=device).view(1,3,1,1)
    with torch.no_grad(): # 그래디언트 계산 비활성화
        for i in range(0, n_images, batch_size):
            batch = images[i:i+batch_size]
            # 만약 grayscale이면 3채널로 변환
            if batch.shape[1] == 1:
                batch = batch.repeat(1,3,1,1)
            # 299x299로 업샘플링
            batch = F.interpolate(batch, size=(299,299), mode='bilinear', align_corners=False)
            # 정규화
            batch = (batch - norm_mean) / norm_std
            pred = model(batch) # Inception-v3 모델에 입력하여 특징 추출
            features.append(pred.cpu().numpy()) # 추출된 특징을 CPU로 이동하여 numpy 배열로 변환
    features = np.concatenate(features, axis=0) # 모든 배치의 특징을 하나로 연결
    return features


def calculate_fid(real_images, generated_images, model, batch_size=32, device=device):
    """
    실제 이미지와 생성된 이미지 간의 FID (Fréchet Inception Distance) 계산
    """
    # real_images, generated_images: torch.Tensor, [0,1] 범위
    feat_real = get_inception_features(real_images, model, batch_size, device) # 실제 이미지 특징 추출
    feat_gen  = get_inception_features(generated_images, model, batch_size, device) # 생성된 이미지 특징 추출

    mu_real = np.mean(feat_real, axis=0) # 실제 이미지 특징의 평균 계산
    mu_gen  = np.mean(feat_gen, axis=0) # 생성된 이미지 특징의 평균 계산
    sigma_real = np.cov(feat_real, rowvar=False) # 실제 이미지 특징의 공분산 행렬 계산
    sigma_gen  = np.cov(feat_gen, rowvar=False) # 생성된 이미지 특징의 공분산 행렬 계산

    # Frechet Distance 공식
    diff = mu_real - mu_gen
    # 공분산 행렬의 제곱근 계산
    covmean, _ = sqrtm(sigma_real.dot(sigma_gen), disp=False)
    # 만약 복소수 부분이 생기면 실수부만 취함
    if np.iscomplexobj(covmean):
        covmean = covmean.real

    # FID 계산
    fid = diff.dot(diff) + np.trace(sigma_real + sigma_gen - 2 * covmean)
    return fid

- IS는 "생성된 이미지가 얼마나 선명하고(Quality) 다양한가(Diversity)?"를 측정하는 지표
    - 오직 생성된 이미지만을 보고 평가
    - 점수가 높을수록 좋은 평가

- IS는 두 가지 관점을 종합

    - 품질 (Quality):
        - 생성된 이미지가 '무엇'인지 명확해야 함.
        - 예를 들어, 개를 생성했다면 누가 봐도 '개'처럼 보여야지, 개와 고양이가 섞인 듯한 불분명한 형태여서는 안됨

    - 다양성 (Diversity):
        - 모델이 다양한 종류의 이미지를 만들어내야 함
        - 1000개의 이미지를 생성했는데 모두 똑같은 종류의 '새' 이미지만 있다면 좋은 모델이라고 할수 없음
        - 여러 종류의 새, 개, 고양이, 자동차 등 다양한 이미지를 골고루 생성해야 함

- IS는 이 두 가지(낮은 엔트로피의 품질, 높은 엔트로피의 다양성)를 KL 발산(KL Divergence)을 이용해 하나의 점수로 통합


In [66]:
#@title Inception Prediction 함수 + Inception Score (IS)계산 함수

def get_inception_predictions(images, model, batch_size=32, device=device):
    """
    Inception-v3 모델을 사용하여 이미지의 클래스 예측 확률 추출
    """
    model.eval()
    n_images = images.shape[0]
    preds = []

    # Inception-v3가 ImageNet으로 학습된 평균/표준편차 (정규화용)
    norm_mean = torch.tensor([0.485, 0.456, 0.406], device=device).view(1,3,1,1)
    norm_std  = torch.tensor([0.229, 0.224, 0.225], device=device).view(1,3,1,1)
    with torch.no_grad(): # 그래디언트 계산 비활성화
        for i in range(0, n_images, batch_size):
            batch = images[i:i+batch_size]
            if batch.shape[1] == 1:
                batch = batch.repeat(1,3,1,1)
            batch = F.interpolate(batch, size=(299,299), mode='bilinear', align_corners=False)
            batch = (batch - norm_mean) / norm_std

            # Inception-v3 모델 통과 >> 클래스별 logit(원점수)
            logits = model(batch)           # (N, 1000)>>클래스별 로짓
            p = F.softmax(logits, dim=1)    # softmax적용>>각 클래스 확률로 변환
            preds.append(p.cpu().numpy())
    preds = np.concatenate(preds, axis=0)   # 모든 배치의 예측 확률을 하나로 연경
    return preds

def calculate_inception_score(
        generated_images,      # 생성된 이미지 텐서, 값 범위 [0,1]
        model,                 # 사전학습 Inception-v3 (softmax 출력 유지)
        batch_size: int = 32,  # 추론 시 배치 크기
        splits: int = 10,      # 이미지 분할(split) 개수
        device=device):        # 연산 장치(GPU/CPU)

    # 생성 이미지들의 Inception 분류 확률 계산
    preds = get_inception_predictions(generated_images, model, batch_size=batch_size, device=device)
    N = preds.shape[0]
    split_scores = []

    # 데이터를 "split"개 구간으로 나눠 분할별로 IS 계산
    for i in range(splits):
        # 각 분할 구간
        part = preds[i * (N // splits):(i + 1) * (N // splits), :]
        # 분할 내 전체 이미지에 대한 평균 클래스 확률
        py = np.mean(part, axis=0)
        scores = []

        # 각 이미지에 대해 KL(p(y|x) || p(y)) 계산
        for j in range(part.shape[0]):
            pyx = part[j, :]
            # log(0) 방지를 위한 1e-10추가
            kl = np.sum(pyx * (np.log(pyx + 1e-10) - np.log(py + 1e-10)))
            scores.append(kl)

        # 분할 KL 평균을 지수화(exp) → 분할 IS
        split_is = np.exp(np.mean(scores))
        split_scores.append(split_is)

    # 4) 모든 분할 IS의 평균·표준편차 산출
    inception_score = np.mean(split_scores)
    inception_std   = np.std(split_scores)

    return inception_score, inception_std

In [67]:
#@title 평가지표 확인

# 학습 종료 후 평가 (예: 1000개 샘플 사용)
n_eval = 1024

# 생성 이미지 준비
z = torch.randn(n_eval, latent_dim, device=device) # 평가에 사용할 노이즈 벡터 생성
eval_labels = torch.randint(0, num_classes, (n_eval,), device=device) # 평가에 사용할 랜덤 레이블 생성
gen_imgs = generator(z, eval_labels) # 생성자에 노이즈와 레이블을 입력하여 이미지 생성
gen_imgs = (gen_imgs + 1) / 2.0  # [-1,1] -> [0,1] 범위로 스케일링

# 실제 이미지 준비 (test dataset에서 n_eval개 샘플)
real_imgs_list = []
for imgs, _ in test_loader:
    real_imgs_list.append(imgs)
    if len(real_imgs_list) * imgs.size(0) >= n_eval:
        break
real_imgs = torch.cat(real_imgs_list, 0)[:n_eval].to(device) # 테스트 데이터셋에서 실제 이미지 샘플링
real_imgs = (real_imgs + 1) / 2.0  # [-1,1] -> [0,1] 범위로 스케일링

# FID 계산
fid_score = calculate_fid(real_imgs, gen_imgs, inception_model_fid, batch_size=32, device=device)
# Inception Score 계산
inception_score, inception_std = calculate_inception_score(gen_imgs, inception_model_is, batch_size=32, splits=10, device=device)

print(f"FID score: {fid_score:.4f}")
print(f"Inception Score: Mean = {inception_score:.4f}, Std = {inception_std:.4f}")

/tmp/ipython-input-3338422441.py:47: DeprecationWarning: The `disp` argument is deprecated and will be removed in SciPy 1.18.0.
  covmean, _ = sqrtm(sigma_real.dot(sigma_gen), disp=False)


FID score: 216.4160
Inception Score: Mean = 2.0326, Std = 0.0811


In [68]:
print(inception_model_fid)

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri